---
### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib
import plotly.express as px
import datetime as DT
import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots

import time 
from tqdm.autonotebook import tqdm
from tslearn.clustering import TimeSeriesKMeans

import warnings
warnings.filterwarnings("ignore")

plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tslearn/clustering/kmeans.py:16: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  warnings.warn(


---
### Функция для поиска ключа по значению в словаре 

In [2]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

---

---
### Функция для считывания и подготовки данных

функция принимает на вход 2 csv-файла – справочник товаров и оборот товаров в штуках по дням за 2021 год, функция осуществляет считывание данных в dataframe, удаляет лишние символы в ячейках dataframe, изменяет типы данных и удаляет строки с пропусками

In [8]:
def data_processing(data_csv_file, spravochnik_csv_file):
    # обработка данных величины оборота
    data = pd.read_csv(data_csv_file, sep=';')
    data = data.rename(columns={'Код товара':'product_code', ' Дата':'date', 
                            ' Оборот шт':'turnoverOfGoods', 
                            ' Оборот руб за вычетом скидки':'turnoverWithDisc', 
                            ' Валовый % скидки':'percentOfDisc'}) 
    data['percentOfDisc'] = data['percentOfDisc'].str.replace('%', '') \
                                                    .str.replace(',', '.')
    data['turnoverWithDisc'] = data['turnoverWithDisc'].str.replace(',', '.') \
                                                        .str.replace(' ', '')
    data['turnoverOfGoods'] = data['turnoverOfGoods'].str.replace(' ', '')
    data = data.astype({'percentOfDisc':'float',
                         'turnoverWithDisc':'float', 
                            'date':'datetime64[ns]'})
    for i in range(data.shape[0]):
        try:
            float(data.loc[i]['turnoverOfGoods'])
        except Exception:
            data = data.drop(i, axis=0)
    data = data.reset_index(drop=True)
    data = data.astype({'turnoverOfGoods':'float'})
    data.loc[(data.turnoverOfGoods < 0), 'turnoverOfGoods'] = 0
    data = data[['date', 'product_code', 'turnoverOfGoods', 
                        'percentOfDisc', 'turnoverWithDisc']]

    # обработка справочника товаров
    spravka = pd.read_csv(spravochnik_csv_file, sep=';')
    spravka = spravka.rename(columns={' Категория':'category', 
                                        ' Группа':'group', 
                                        ' Подгруппа':'subgroup', 
                                        'Код товара':'product_code'})
    spravka = spravka[['product_code', 'category', 'group', 'subgroup']]
    main_data = data.merge(spravka, how='left', on='product_code')
    main_data['category'] = main_data['category'].str.replace('[', '').str.replace(']','')
    main_data = main_data[main_data['date'].dt.year == 2021]
    return main_data

In [9]:
main_data = data_processing('Данные о продажах.csv', 'Справочник товаров.csv')

In [5]:
main_data2 = data_processing('Данные о продажах.csv', 'Справочник товаров.csv')

---

---
### Функция для исключения влияния оборота 15го числа каждого месяца (регулярная акция) на изменение динамики спроса 

на вход функции подается dataframe, полученный в результате запуска предыдущей функции, данная функция заменяет величину оборота для каждого товара за 15ое числа месяца на среднедневное значение за месяц (без учета оборота за 15ое число)

In [12]:
def exclude_infl_15(data):
    # обработка данных величины оборота в дни 
    # проведения регулярной акции (15 числа каждого месяца)
    data['month'] = data['date'].dt.month
    isk_15 = data[data['date'].dt.day != 15] \
            .groupby(['product_code', 'month'], 
                    as_index=False) \
                .agg({'turnoverOfGoods':'mean'})
    prod_lst = list(isk_15['product_code'].unique())
    isk_15['turnoverOfGoods'] = round(isk_15['turnoverOfGoods'])
    add_lst = isk_15.groupby('product_code', as_index=False) \
                    .agg({'month':'count'}) \
                        .query('month != 12')['product_code'] \
                            .to_list()
    data = data.query('product_code not in @add_lst') \
                .reset_index(drop=True)
    for i in prod_lst:
        for j in range(1, 13):
            if data[(data['product_code'] == i) \
                & (data['date'].dt.month == j) \
                & (data['date'].dt.day == 15)].shape[0] != 0:
                data.loc[((data.product_code == i) \
                    & (data.date.dt.day == 15) \
                    & (data.date.dt.month == j)), 
                    'turnoverOfGoods'] = float(isk_15[(isk_15['product_code'] == i) \
                        & (isk_15['month'] == j)]['turnoverOfGoods'])
    add_lst = data.groupby(['product_code', 'month'],as_index=False) \
                    .agg({'turnoverOfGoods':'sum'}) \
                    .groupby('product_code',as_index=False) \
                    .agg({'month':'count'}) \
                    .query('month != 12')['product_code'].to_list()
    data = data.query('product_code not in @add_lst')
    return data

In [13]:
data = exclude_infl_15(main_data)

In [14]:
data2 = exclude_infl_15(main_data2)

---

---
### Функция для принятия запроса и получения данных из сформированного dataframe (запрос подается в excel-файле)

на вход функции подается запрос в виде excel-файла, в котором указывается код товара, подгруппа, группа или категория товаров, и dataframe, полученный в результате запуска предыдущей функции, данная функция отбирает необходимые строки dataframe в соответствии с запросом

In [15]:
def get_data(file_excel, data):
    # получение данных величины оборота 
    # по запрашиваемому товару/группе/подгруппе/категории
    workbook = xlrd.open_workbook(file_excel)
    worksheet = workbook.sheet_by_index(0)
    good = worksheet.cell_value(0, 0)
    # если запрашивается товар по коду
    if type(good) == float:
        good = int(good)
        data = data[data['product_code'] == good]
        data = data.reset_index(drop=True)
        if data.shape[0] == 0:
            return 'Группа товаров не найдена в базе!'
    else:
        # если запрашивается категория
        if data[data['category'] == good].shape[0] != 0:
            data = data[data['category'] == good]
            data = data.reset_index(drop=True)
        # если запрашивается группа
        elif data[data['group'] == good].shape[0] != 0:
            data = data[data['group'] == good]
            data = data.reset_index(drop=True)
        # если запрашивается подгруппа
        elif data[data['subgroup'] == good].shape[0] != 0:
            data = data[data['subgroup'] == good]
            data = data.reset_index(drop=True)
        # если данные по искомому товару не найдены в базе
        else:
            return 'Группа товаров не найдена в базе!'
    return data

In [16]:
nes_data = get_data("Запрашиваемый товар.xlsx", data)

In [17]:
nes_data2 = get_data("Запрашиваемый товар.xlsx", data2)

---

---
### Функция для расчета и визуализации описательных статистик ряда и общей информации об анализируемых данных

на вход функции подается dataframe, полученный в результате запуска предыдущей функции, данная функция предоставляет информацию об описательных статистиках ряда в табличном и графическом виде

In [18]:
def get_statistics(data):
    # расчет и визуализация описательных статистик 
    # ряда динамики величины оборота
    month_lst = ['Январь', 'Февраль', 'Март', 
                'Апрель', 'Май', 'Июнь', 'Июль', 
                'Август', 'Сентябрь', 'Октябрь', 
                'Ноябрь', 'Декабрь']
    lst_names = []
    lst_values = []
    # для одиночного товара
    if data['product_code'].nunique() == 1:
        lst_names.append('Товар')
        lst_names.append('Подгруппа')
        lst_names.append('Группа')
        lst_names.append('Категория')
        lst_values.append(data["product_code"].unique()[0])
        lst_values.append(data["subgroup"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["group"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["category"].unique()[0].replace("[", "").replace("]", ""))

    # для подгруппы товаров
    elif data['subgroup'].nunique() == 1:
        lst_names.append('Подгруппа')
        lst_names.append('Группа')
        lst_names.append('Категория')
        lst_names.append('Число наименований товаров')
        lst_values.append(data["subgroup"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["group"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["category"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["product_code"].nunique())

    # для группы товаров
    elif data['group'].nunique() == 1:
        lst_names.append('Группа')
        lst_names.append('Категория')
        lst_names.append('Число наименований товаров')
        lst_names.append('Число подгрупп')
        lst_values.append(data["group"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["category"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["product_code"].nunique())
        lst_values.append(data["subgroup"].nunique())
        
    # для категории товаров
    elif data['category'].nunique() == 1:
        lst_names.append('Категория')
        lst_names.append('Число наименований товаров')
        lst_names.append('Число подгрупп')
        lst_names.append('Число групп')
        lst_values.append(data["category"].unique()[0].replace("[", "").replace("]", ""))
        lst_values.append(data["product_code"].nunique())
        lst_values.append(data["subgroup"].nunique())
        lst_values.append(data["group"].nunique())
    df_agg_m = data[["date", "turnoverOfGoods"]].resample("M", on="date") \
                                                .sum().reset_index()
    lst_names.append('Среднемесячный оборот, шт')
    lst_names.append('Среднеквадратическое отклонение, шт')
    lst_names.append('Минимальное значение оборота, шт')
    lst_names.append('Максимальное значение оборота, шт')
    lst_names.append('Медианное значение оборота, шт')
    lst_names.append('Первый квартиль величины оборота,шт')    
    lst_names.append('Третий квартиль величины оборота, шт')
    lst_names.append('Коэффициент вариации, %')
    lst_values.append(round(df_agg_m['turnoverOfGoods'].describe()['mean'])) 
    lst_values.append(int(df_agg_m['turnoverOfGoods'].describe()['std']))
    lst_values.append(int(df_agg_m['turnoverOfGoods'].describe()['min'])) 
    lst_values.append(int(df_agg_m['turnoverOfGoods'].describe()['max']))   
    lst_values.append(int(df_agg_m['turnoverOfGoods'].describe()['50%'])) 
    lst_values.append(int(df_agg_m['turnoverOfGoods'].describe()['25%']))  
    lst_values.append(int(df_agg_m['turnoverOfGoods'].describe()['75%'])) 
    lst_values.append(int(df_agg_m['turnoverOfGoods'].describe()['std'] / df_agg_m['turnoverOfGoods'] \
                                                        .describe()['mean'] * 100))
    df = pd.DataFrame({'0':lst_names, '1':lst_values})
    fig = go.Figure(data=[go.Table(
            header=dict(values=['',''],
                        fill_color='rgb(102, 194, 165)',
                        align='center',
                        line_color='darkslategray'),
            cells=dict(values=[df['0'], df['1']],
                    fill_color=['white', 'white'],
                    line_color='darkslategray',
                    align=['left', 'center'], font=dict(size=13), height=20))
        ])
    fig.show()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=month_lst, y=df_agg_m['turnoverOfGoods'],
                    mode='lines+markers',
                    name='', marker_color='rgb(102, 194, 165)'))
    fig.update_layout( 
                yaxis_title_text='Оборот, шт', 
                showlegend = False, template="simple_white", 
                height=600, width=1000, yaxis_tickformat = 'f.'
                    )
    if nes_data['product_code'].nunique() == 1:
        fig.update_layout(title_text=f'Динамика величины оборота товара "{nes_data["product_code"].unique()[0].replace("[", "").replace("]", "")}"')
    elif nes_data['subgroup'].nunique() == 1:
        fig.update_layout(title_text=f'Динамика величины оборота подгруппы товаров "{nes_data["subgroup"].unique()[0].replace("[", "").replace("]", "")}"')
    elif nes_data['group'].nunique() == 1:
        fig.update_layout(title_text=f'Динамика величины оборота группы товаров "{nes_data["group"].unique()[0].replace("[", "").replace("]", "")}"')
    elif nes_data['category'].nunique() == 1:
        fig.update_layout(title_text=f'Динамика величины оборота категории товаров "{nes_data["category"].unique()[0].replace("[", "").replace("]", "")}"')
    fig.show()
    y = df_agg_m['turnoverOfGoods'].values
    lowerfence = min(y)
    q1, median, q3 = np.percentile(y, 25), np.percentile(y, 50), np.percentile(y, 75)
    upperfence = max([y0 for y0 in y if y0 < (q3 + 1.5*(q3-q1))])
    fig = go.Figure()
    fig.add_trace(go.Box(
                    x=[''],
                    q1=[round(q1)], median=[round(median)], mean=[round(np.mean(y))],
                    q3=[round(q3)], lowerfence=[lowerfence],
                    upperfence=[upperfence], orientation='v', showlegend=False,
                    name="",
                    boxpoints='all',
                    marker_color='rgb(102, 194, 165)',
                    line_color='rgb(102, 194, 165)', hoverinfo='y'
                        ))
    fig.update_layout( 
                    xaxis_title_text='', 
                    yaxis_title_text='Оборот, шт', 
                    showlegend = False, template="simple_white", 
                    height=600, width=800, yaxis_tickformat = 'f.'
                    )
    if data['product_code'].nunique() == 1:
        fig.update_layout(title_text=f'Диаграмма размаха величины оборота товара "{data["product_code"].unique()[0].replace("[", "").replace("]", "")}"')
    elif data['subgroup'].nunique() == 1:
        fig.update_layout(title_text=f'Диаграмма размаха величины оборота подгруппы товаров "{data["subgroup"].unique()[0].replace("[", "").replace("]", "")}"')
    elif data['group'].nunique() == 1:
        fig.update_layout(title_text=f'Диаграмма размаха величины оборота группы товаров "{data["group"].unique()[0].replace("[", "").replace("]", "")}"')
    elif data['category'].nunique() == 1:
        fig.update_layout(title_text=f'Диаграмма размаха величины оборота категории товаров "{data["category"].unique()[0].replace("[", "").replace("]", "")}"')
    fig.show()

        

In [19]:
get_statistics(nes_data)

TypeError: string indices must be integers

---

---
### Функция для определения порогового значения коэффициента сезонности для выявления в ряду ярко выраженных сезонных колебаний

на вход функциии подается запрос в виде excel-файла, dataframe с данными о продажах всех товаров, а также процентиль,
данная функция рассчитывает месячные коэффициенты сезонности для каждого товара запрашиваемой категории, строит распределение из рассчитанных значений и в качестве порогового значения берет заданный процентиль распределения

In [ ]:
def find_seas_threshold(file_excel, data, percent=75):
    # определение порогового значения коэффициента сезонности
    workbook = xlrd.open_workbook(file_excel)
    worksheet = workbook.sheet_by_index(0)
    good = worksheet.cell_value(0, 0)
    if type(good) == float:
        good = int(good)
        data2 = data[data['product_code'] == good]
        cat = data2['category'].unique()[0]
        data2 = data.reset_index(drop=True)
        if data2.shape[0] == 0:
            return 'Группа товаров не найдена в базе!'
    else:
        # если запрашивается категория
        if data[data['category'] == good].shape[0] != 0:
            data2 = data[data['category'] == good]
            data2 = data2.reset_index(drop=True)
            cat = data2['category'].unique()[0]
        # если запрашивается группа
        elif data[data['group'] == good].shape[0] != 0:
            data2 = data[data['group'] == good]
            data2 = data2.reset_index(drop=True)
            cat = data2['category'].unique()[0]
        # если запрашивается подгруппа
        elif data[data['subgroup'] == good].shape[0] != 0:
            data2 = data[data['subgroup'] == good]
            data2 = data2.reset_index(drop=True)
            cat = data2['category'].unique()[0]
        # если данные по искомому товару не найдены в базе
        else:
            return 'Группа товаров не найдена в базе!'
    # определяем категорию, для которой будем считать порог сезонности
    cat_df = data[data['category'] == cat]
    cat_df_gr = cat_df.groupby(['product_code', 'month'], as_index=False) \
        .agg({'turnoverOfGoods':'sum'})
    cat_df_gr['mean'] = 0
    prod_lst = list(cat_df_gr['product_code'].unique())
    # определяем для каждого товара для каждого месяца коэффициент сезонности
    for i in prod_lst:
        mean_turn = cat_df_gr[cat_df_gr['product_code'] == i]['turnoverOfGoods'].mean()
        cat_df_gr.loc[(cat_df_gr.product_code == i), 'mean'] = mean_turn
    cat_df_gr['seasonality'] = cat_df_gr['turnoverOfGoods'] / cat_df_gr['mean']
    # строим распределение значений коэффициентов сезонности
    perc_seas = round(np.percentile(cat_df_gr['seasonality'], percent), 2)
    fig = go.Figure()
    fig.add_trace(go.Histogram(
                                x=cat_df_gr[cat_df_gr['seasonality'] < 5]['seasonality'],
                                histnorm='percent',
                                name='control', 
                                marker_color='rgb(102, 194, 165)',
                                opacity=0.5, xbins = dict(size = 0.1), 
                                ))
    fig.add_vline(x=perc_seas, line_width=2.5, line_dash="dash", line_color="black", 
                    annotation_text=f"Пороговое значение={str(perc_seas).replace('.', ',')}", 
                    annotation_position="top", annotation_font_size=15, 
                    annotation_font_color="black")
    fig.update_layout(title_text='Гистограмма распределения коэффициентов сезонности',
                        xaxis_title_text='Коэффициент сезонности', 
                        yaxis_title_text='Частота, %', 
                        bargap=0, 
                        bargroupgap=0, showlegend = False, 
                        template="simple_white", height=500)
    fig.show()

In [ ]:
find_seas_threshold("Запрашиваемый товар.xlsx", nes_data)

---

---
### Функция для выявления в ряду ярко выраженных сезонных колебаний

на вход функции подается dataframe с данными величины оборота анализируемого товара, подгруппы, группы или категории, а также пороговое значение коэффициента сезонности, найденное на предыдущем шаге, данная функция выдает информацию в текстовом, табличном и графическом виде о том, в каких месяцах наблюдается повышенный и пониженный спрос

In [ ]:
def find_season(data, threshold=1.15):
    month_lst = ['Январь', 'Февраль', 'Март', 
                'Апрель', 'Май', 'Июнь', 'Июль', 
                'Август', 'Сентябрь', 'Октябрь', 
                'Ноябрь', 'Декабрь']
    data_agg_d = data[['date', 'turnoverOfGoods']] \
                    .resample('D', on='date') \
                        .sum().reset_index()
    data_agg_m = data[['date', 'turnoverOfGoods']] \
                    .resample('M', on='date') \
                        .sum().reset_index()
    data_agg_m['date'] = month_lst
    data_agg_m['mean'] = data_agg_m['turnoverOfGoods'].mean()
    data_agg_m['seasonality'] = data_agg_m['turnoverOfGoods'] / data_agg_m['mean']
    seas_month_high = []
    seas_month_low = []
    for i in range(data_agg_m.shape[0]):
        if data_agg_m.loc[i]['seasonality'] > threshold:
            seas_month_high.append(i)
        if data_agg_m.loc[i]['seasonality'] < (1 - (threshold - 1)):
            seas_month_low.append(i)
    if len(seas_month_high) != 0 or len(seas_month_low) != 0:
        print('В ряду присутствуют ярко выраженные сезонные колебания')
    if len(seas_month_high) != 0:
        print(f'Месяц повышенного спроса – {", ".join([data_agg_m.loc[j]["date"] for j in seas_month_high])}')
        for i in seas_month_high:
            month = data_agg_m.loc[i]['date']
            if month.endswith('ь') or month.endswith('й'):
                month = ''.join([month[:-1], 'е'])
            else:
                month = ''.join([month, 'е'])
            day_max = str(data_agg_d[(data_agg_d['date'].dt.month == i+1) \
                                    & (data_agg_d['date'].dt.day != 15)] \
                                        .set_index('date')['turnoverOfGoods'] \
                                            .idxmax().day)
            print(f'Максимальная величина спроса на данную группу товаров в {month} наблюдалась {day_max} числа')
            print(f'Отклонение величины оборота в {month} от средней величины оборота за год составило {round((data_agg_m.loc[i]["seasonality"] - 1) * 100, 2)} %')
    if len(seas_month_low) != 0:
        print(f'Месяц пониженного спроса – {", ".join([data_agg_m.loc[j]["date"] for j in seas_month_low])}')
        for i in seas_month_low:
            month = data_agg_m.loc[i]['date']
            if month.endswith('ь') or month.endswith('й'):
                month = ''.join([month[:-1], 'е'])
            else:
                month = ''.join([month, 'е'])
            day_min = str(data_agg_d[(data_agg_d['date'].dt.month == i+1) & (data_agg_d['date'].dt.day != 15)] \
                                                                                .set_index('date')['turnoverOfGoods'] \
                                                                                    .idxmin().day)
            print(f'Минимальная величина спроса на данную группу товаров в {month} наблюдалась {day_min} числа')
            print(f'Отклонение величины оборота в {month} от средней величины оборота за год составило {round((1 - data_agg_m.loc[i]["seasonality"]) * 100, 2)} %')
    print('_____________________________________________________________________________________________')
    df = data_agg_m
    df['otklonenie'] = df['seasonality'] - 1
    text_lst = []
    y_lst = []
    month_lst = df['date'].to_list()
    color_lst = []
    threshold = (threshold - 1) * 100
    for i in df['otklonenie'].to_list():
        y_lst.append(round(i, 4) * 100)
        if abs(i) * 100 > threshold:
            if i < 0:
                text_lst.append(format(i * 100, '.2f'))
                color_lst.append('crimson')
            else:
                text_lst.append("+" + (format(i * 100, '.2f')))
                color_lst.append('green')
        elif i < 0:
            text_lst.append(format(i * 100, '.2f'))
            color_lst.append('lightgray')
        else:
            text_lst.append("+" + (format(i * 100, '.2f')))
            color_lst.append('lightgray')
    fig = go.Figure()
    fig.add_trace(go.Bar(x=month_lst, y=y_lst,base=0,
                            marker_color=color_lst, text=text_lst, 
                            textposition='outside'))
    fig.update_layout(title = f"Отклонение месячного оборота от средней величины за {data['date'].dt.year.loc[0]} год", 
                        showlegend = False, template="simple_white", height=500, yaxis_title="Отклонение оборота в %",)
    fig.add_hline(y=threshold, line_width=2, line_dash="dash", line_color="black", 
                    annotation_text=f"Пороговое значение={int(threshold)}%", 
                    annotation_position="bottom left")
    fig.add_hline(y=-threshold, line_width=2, line_dash="dash", line_color="black", 
                    annotation_text=f"Пороговое значение={int(threshold)}%", 
                    annotation_position="bottom left")
    fig.add_hline(y=0, line_width=1, line_color="black")
    fig.show()
    df['turnoverOfGoods'] = pd.Series(["{0:.0f}".format(val) for val in df['turnoverOfGoods']], index = df.index)
    
    df = df.loc[:, :'seasonality'].rename({'date':'Месяц', 'turnoverOfGoods':'Оборот, шт', 'seasonality':'Отклонение, %'}, 
                            axis=1)[['Месяц', 'Оборот, шт', 'Отклонение, %']]
    df['Отклонение, %'] = df['Отклонение, %'] * 100 - 100
    df['Отклонение, %'] = pd.Series(["{0:+.2f}".format(val) for val in df['Отклонение, %']])
    df['Отклонение, %'] = df['Отклонение, %'].str.replace(".", ",")
    
    color_lst = []
    for i in range(12):
        if i in seas_month_high:
            color_lst.append('green')
        elif i in seas_month_low:
            color_lst.append('red')
        else:
            color_lst.append('white')

    fig = go.Figure(data=[go.Table(
            header=dict(values=list(df.columns),
                        fill_color='rgb(204, 204, 204)',
                        align='center',
                        line_color='darkslategray', font=dict(color='black', size=12)),
            cells=dict(values=[df['Месяц'], df['Оборот, шт'], df['Отклонение, %']],
                    fill_color=[color_lst, 'white', 'white'],
                    line_color='darkslategray',
                    align=['center', 'right', 'right'], font=dict(size=12)))
        ])
    fig.show()
        

In [ ]:
find_season(nes_data)

В ряду присутствуют ярко выраженные сезонные колебания
Месяц повышенного спроса – Сентябрь, Октябрь, Декабрь
Максимальная величина спроса на данную группу товаров в Сентябре наблюдалась 10 числа
Отклонение величины оборота в Сентябре от средней величины оборота за год составило 16.52 %
Максимальная величина спроса на данную группу товаров в Октябре наблюдалась 10 числа
Отклонение величины оборота в Октябре от средней величины оборота за год составило 31.8 %
Максимальная величина спроса на данную группу товаров в Декабре наблюдалась 31 числа
Отклонение величины оборота в Декабре от средней величины оборота за год составило 40.68 %
Месяц пониженного спроса – Февраль, Июнь, Июль
Минимальная величина спроса на данную группу товаров в Феврале наблюдалась 7 числа
Отклонение величины оборота в Феврале от средней величины оборота за год составило 16.63 %
Минимальная величина спроса на данную группу товаров в Июне наблюдалась 27 числа
Отклонение величины оборота в Июне от средней величины оборо

---

---
### Функция для визуализации ярко выраженных сезонных колебаний

на вход функции подается dataframe с данными величины оборота анализируемого товара, подгруппы, группы или категории, а также пороговое значение коэффициента сезонности, найденное с помощью функции find_seas_threshold, данная функция выводит графики с отметкой месяцев пониженного и повышенного спроса на товары, а также отметкой пикового спроса внутри месяцев пониженного и повышенного спроса

In [ ]:
def find_peak(data, threshold=1.15):
    month_lst = ['Январь', 'Февраль', 'Март', 'Апрель', 
                'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 
                'Октябрь', 'Ноябрь', 'Декабрь']
    data_agg_d = data[['date', 'turnoverOfGoods']].resample('D', on='date').sum().reset_index()
    data_agg_m = data[['date', 'turnoverOfGoods']].resample('M', on='date').sum().reset_index()
    data_agg_m['date'] = month_lst
    data_agg_m['mean'] = data_agg_m['turnoverOfGoods'].mean()
    data_agg_m['seasonality'] = data_agg_m['turnoverOfGoods'] / data_agg_m['mean']
    seas_month_high = []
    seas_month_low = []
    for i in range(data_agg_m.shape[0]):
        if data_agg_m.loc[i]['seasonality'] > threshold:
            seas_month_high.append(i)
        if data_agg_m.loc[i]['seasonality'] < (1 - (threshold - 1)):
            seas_month_low.append(i)
    
    x1_lst = [data_agg_m.loc[i]['date'] for i in seas_month_low]
    x2_lst = [data_agg_m.loc[i]['date'] for i in seas_month_high]
    y1_lst = [data_agg_m.loc[i]['turnoverOfGoods'] for i in seas_month_low]
    y2_lst = [data_agg_m.loc[i]['turnoverOfGoods'] for i in seas_month_high]

    #график динамики оборота по месяцам
    fig = go.Figure(data = go.Scatter(name='Оборот фактический', x=data_agg_m['date'], 
                            y=data_agg_m['turnoverOfGoods'],
                            mode='lines',
                            line = dict(color='gray', width=2)), 
                            layout_yaxis_range=[0, max(y2_lst) + max(y2_lst) * 0.1])
    fig.update_layout(showlegend = True, template="simple_white", 
                    height=600, width=1000, yaxis_tickformat = 'f.',     
                    legend=dict(yanchor="bottom", y=0.9, xanchor="right", x=0.9),    
                    yaxis_title="Оборот, шт", xaxis_title="День")
    for i, j in zip(x2_lst, y2_lst):
        fig.add_trace(go.Scatter(name='', x=[i, i], 
                            y=[j, 0],
                            mode='lines',
                            line = dict(color='gray', width=2, dash='dash'),
                            showlegend=False))
    for i, j in zip(x1_lst, y1_lst):
        fig.add_trace(go.Scatter(name='', x=[i, i], 
                            y=[j, 0],
                            mode='lines',
                            line = dict(color='gray', width=2, dash='dash'),    
                            showlegend=False))
    fig.add_trace(go.Scatter(name='Сезонный минимум', x=x1_lst, 
                            y=y1_lst,
                            mode='markers+text',
                            marker=dict(size=10,
                            color='red'), 
                            text=[int(i) for i in y1_lst],
                            textposition="top center"))
    fig.add_trace(go.Scatter(name='Сезонный максимум', x=x2_lst, 
                            y=y2_lst,
                            mode='markers+text',
                            marker=dict(size=10,
                            color='green'),   
                            text=[int(i) for i in y2_lst],
                            textposition="top center"))
    if data['product_code'].nunique() == 1:
        fig.update_layout(title_text=f'Динамика величины оборота товара "{data["product_code"].unique()[0].replace("[", "").replace("]", "")}" по месяцам {data["date"].dt.year[0]} года')
    elif data['subgroup'].nunique() == 1:
        fig.update_layout(title_text=f'Динамика величины оборота подгруппы товаров "{data["subgroup"].unique()[0].replace("[", "").replace("]", "")}" по месяцам {data["date"].dt.year[0]} года')
    elif data['group'].nunique() == 1:
        fig.update_layout(title_text=f'Динамика величины оборота группы товаров "{data["group"].unique()[0].replace("[", "").replace("]", "")}" по месяцам {data["date"].dt.year[0]} года')
    else:
        fig.update_layout(title_text=f'Динамика величины оборота категории товаров "{data["category"].unique()[0].replace("[", "").replace("]", "")}" по месяцам {data["date"].dt.year[0]} года')
    fig.show()

    # дневные графики для сезонных минимумов    
    for i in seas_month_low:
        data_d_min = data_agg_d[data_agg_d['date'].dt.month == i + 1] 
        month = data_agg_m.loc[i]['date']
        if month.endswith('ь') or month.endswith('й'):
            month = ''.join([month[:-1].lower(), 'е'])
        else:
            month = ''.join([month.lower(), 'е'])   
        fig = go.Figure(data = go.Scatter(name='Оборот фактический', x=[i.strftime('%-d') for i in data_d_min['date']], 
                            y=data_d_min['turnoverOfGoods'],
                            mode='lines',
                            line = dict(color='gray', width=2)), 
                            layout_yaxis_range=[0, data_d_min['turnoverOfGoods'].max() + data_d_min['turnoverOfGoods'].max() * 0.1])
        fig.update_layout(showlegend = True, template="simple_white", 
                    height=600, width=1000, yaxis_tickformat = 'f.',     
                    legend=dict(yanchor="bottom", y=0.9, xanchor="right", x=0.9),   
                    yaxis_title="Оборот, шт", xaxis_title="День")
        if data['product_code'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота товара "{data["product_code"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        elif data['subgroup'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота подгруппы товаров "{data["subgroup"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        elif data['group'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота группы товаров "{data["group"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        else:
            fig.update_layout(title_text=f'Динамика величины оборота категории товаров "{data["category"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        x_lst = data_d_min[data_d_min['turnoverOfGoods'] == data_d_min['turnoverOfGoods'].min()]['date'].to_list()
        y_lst = [data_d_min['turnoverOfGoods'].min()]
        for i in range(len(x_lst)-1):
            y_lst.append(y_lst[0])
        for i, j in zip(x_lst, y_lst):
            fig.add_trace(go.Scatter(name='', x=[i.strftime('%-d'), i.strftime('%-d')], 
                            y=[j, 0],
                            mode='lines',
                            line = dict(color='gray', width=2, dash='dash'),    
                            showlegend=False))
        fig.add_trace(go.Scatter(name='Минимальная величина оборота', x=[i.strftime('%-d') for i in x_lst], 
                            y=y_lst,
                            mode='markers+text',
                            marker=dict(size=10,
                            color='red'), 
                            text=[int(i) for i in y_lst],
                            textposition="top center"))
        fig.show()  

    # дневные графики для сезонных максимумов   
    for i in seas_month_high:
        data_d_max = data_agg_d[data_agg_d['date'].dt.month == i + 1] 
        month = data_agg_m.loc[i]['date']
        if month.endswith('ь') or month.endswith('й'):
            month = ''.join([month[:-1].lower(), 'е'])
        else:
            month = ''.join([month, 'е'])   
        fig = go.Figure(data = go.Scatter(name='Оборот фактический', x=[i.strftime('%-d') for i in data_d_max['date']], 
                            y=data_d_max['turnoverOfGoods'],
                            mode='lines',
                            line = dict(color='gray', width=2)), 
                            layout_yaxis_range=[0, data_d_max['turnoverOfGoods'].max() + data_d_max['turnoverOfGoods'].max() * 0.1])
        fig.update_layout(showlegend = True, template="simple_white", 
                    height=600, width=1000, yaxis_tickformat = 'f.',     
                    legend=dict(yanchor="bottom", y=0.97, xanchor="right", x=0.9),    
                    yaxis_title="Оборот, шт", xaxis_title="День")
        if data['product_code'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота товара "{data["product_code"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        elif data['subgroup'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота подгруппы товаров "{data["subgroup"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        elif data['group'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота группы товаров "{data["group"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        else:
            fig.update_layout(title_text=f'Динамика величины оборота категории товаров "{data["category"].unique()[0].replace("[", "").replace("]", "")}" в {month}')
        x_lst = data_d_max[data_d_max['turnoverOfGoods'] == data_d_max['turnoverOfGoods'].max()]['date'].to_list()
        y_lst = [data_d_max['turnoverOfGoods'].max()]
        for i in range(len(x_lst)-1):
            y_lst.append(y_lst[0])
        for i, j in zip(x_lst, y_lst):
            fig.add_trace(go.Scatter(name='', x=[i.strftime('%-d'), i.strftime('%-d')], 
                            y=[j, 0],
                            mode='lines',
                            line = dict(color='gray', width=2, dash='dash'),    
                            showlegend=False))
        fig.add_trace(go.Scatter(name='Максимальная величина оборота', x=[i.strftime('%-d') for i in x_lst], 
                            y=y_lst,
                            mode='markers+text',
                            marker=dict(size=10,
                            color='green'), 
                            text=[int(i) for i in y_lst],
                            textposition="top center"))
        fig.show()

In [ ]:
find_peak(nes_data)

---

---
### Функция для прогнозирования величины оборота товаров

на вход функции подается dataframe с данными величины оборота анализируемого товара, подгруппы, группы или категории, словарь с моделями, перебор которых необходимо осуществить, вспомогательная переменная, которая далее будет представлять из себя dataframe с рассчитанными метриками точности предсказаний моделей, а также горизонт прогнозирования (по умолчанию 2 месяца), вывод функции включает график, на котором изображается исходный ряд, ряд предсказанных значений оборота, а также линия тренда, и рассчитанные значения метрик точности предсказаний моделей в табличном виде

In [ ]:
models_dct = {'linear add':0, 'linear mul':0, 'polynomial2 add':0, 'polynomial2 mul':0,   
                'polynomial3 add':0, 'polynomial3 mul':0, 'log add':0, 'log mul':0, 
                'exp add':0, 'exp mul':0, 'pow add':0, 'pow mul':0}

In [ ]:
def find_model_predskaz(data, models_dct, df=0, n=2):
    # подбор наилучшей модели ряда динамики оборота
    models_lst = list(models_dct.keys())
    r2_lst = []
    mape_lst = []
    mae_lst = []
    mse_lst = []
    rmse_lst = []
    tr_lst = []
    seas_lst = []
    data_agg_m = data[['date', 'turnoverOfGoods']].resample('M', on='date') \
                                                    .sum().reset_index()
    ls_time = [i for i in range(1, 25)]
    data_agg_m = pd.concat([data_agg_m[:12], data_agg_m[24:]])
    data_agg_m['date'] = ls_time
    data_agg_m = data_agg_m.reset_index(drop=True)

    x = data_agg_m['date'].to_list()
    y = data_agg_m['turnoverOfGoods'].to_list()

    for i in models_lst:
        data_agg_m2 = data_agg_m.copy()
        if 'linear' in i:
            k = np.polyfit(x, y, 1)[0]
            b = np.polyfit(x, y, 1)[1]
            lst = []
            for j in x:
                lst.append(k * j + b)
            data_agg_m2['trend'] = lst
            if 'add' in i:
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] - data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 
                                        'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                            'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 
                                        'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2['total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] - data_agg_m2['total_ind']/12
                data_agg_m2['model_data'] = data_agg_m2['trend'] + data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['linear add'] = r2_score(data_agg_m2['turnoverOfGoods'], 
                                                    data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], 
                                                    data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs(
                    (data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                    ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], 
                                                        data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                            data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('линейный')
                seas_lst.append('аддитивная')
            else:
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] / data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 
                                    'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                        'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 
                                    'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2.loc[:11, 'total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] / (
                                                    data_agg_m2['total_ind']/12)
                data_agg_m2['model_data'] = data_agg_m2['trend'] * data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['linear mul'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs(
                                    (data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], 
                                                        data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                        data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('линейный')
                seas_lst.append('мультипликативная')
        elif 'polynomial2' in i:
            a = np.polyfit(x, y, 2)[0]
            b = np.polyfit(x, y, 2)[1]
            c = np.polyfit(x, y, 2)[2]
            if 'add' in i:
                lst = []
                for j in x:
                    lst.append(a * j**2 + b * j + c)
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] - data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 
                                    'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                    'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 
                                        'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2['total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] - data_agg_m2['total_ind']/12
                data_agg_m2['model_data'] = data_agg_m2['trend'] + data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['polynomial2 add'] = r2_score(data_agg_m2['turnoverOfGoods'], 
                                                            data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], 
                                                        data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                        data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('полином 2й степени')
                seas_lst.append('аддитивная')
            else:
                lst = []
                for j in x:
                    lst.append(a * j**2 + b * j + c)
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] / data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2.loc[:11, 'total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] / (data_agg_m2['total_ind']/12)
                data_agg_m2['model_data'] = data_agg_m2['trend'] * data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['polynomial2 mul'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('полином 2й степени')
                seas_lst.append('мультипликативная')
        elif 'polynomial3' in i:
            a = np.polyfit(x, y, 3)[0]
            b = np.polyfit(x, y, 3)[1]
            c = np.polyfit(x, y, 3)[2]
            d = np.polyfit(x, y, 3)[3]
            if 'add' in i:
                lst = []
                for j in x:
                    lst.append(a * j**3 + b * j**2 + c * j + d)
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] - data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                            'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2['total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] - data_agg_m2['total_ind']/12
                data_agg_m2['model_data'] = data_agg_m2['trend'] + data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['polynomial3 add'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('полином 3й степени')
                seas_lst.append('аддитивная')
            else:
                lst = []
                for j in x:
                    lst.append(a * j**3 + b * j**2 + c * j + d)
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] / data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2.loc[:11, 'total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] / (data_agg_m2['total_ind']/12)
                data_agg_m2['model_data'] = data_agg_m2['trend'] * data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['polynomial3 mul'] = r2_score(data_agg_m2['turnoverOfGoods'], 
                                                            data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('полином 3й степени')
                seas_lst.append('мультипликативная')
        elif 'log' in i:
            a = np.polyfit(np.log(x), y, 1)[0]
            b = np.polyfit(np.log(x), y, 1)[1]
            if 'add' in i:
                lst = []
                for j in x:
                    lst.append(a * np.log(j) + b)
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] - data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2['total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] - data_agg_m2['total_ind']/12
                data_agg_m2['model_data'] = data_agg_m2['trend'] + data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['log add'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('логарифмический')
                seas_lst.append('аддитивная')
            else:
                lst = []
                for j in x:
                    lst.append(a * np.log(j) + b)
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] / data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2.loc[:11, 'total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] / (data_agg_m2['total_ind']/12)
                data_agg_m2['model_data'] = data_agg_m2['trend'] * data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['log mul'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('логарифмический')
                seas_lst.append('мультипликативная')
        elif 'exp' in i:
            k = np.polyfit(x, np.log(y), 1)[0]
            ln_b = np.polyfit(x, np.log(y), 1)[1]
            b = np.exp(ln_b)
            if 'add' in i:
                lst = []
                for j in x:
                    lst.append(b * np.exp(k * j))
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] - data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2['total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] - data_agg_m2['total_ind']/12
                data_agg_m2['model_data'] = data_agg_m2['trend'] + data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['exp add'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('экспоненциальный')
                seas_lst.append('аддитивная')
            else:
                lst = []
                for j in x:
                    lst.append(b * np.exp(k * j))
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] / data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                    'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2.loc[:11, 'total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] / (data_agg_m2['total_ind']/12)
                data_agg_m2['model_data'] = data_agg_m2['trend'] * data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['exp mul'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                                ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('экспоненциальный')
                seas_lst.append('мультипликативная')
        else:
            k = np.polyfit(np.log(x), np.log(y), 1)[0]
            ln_b = np.polyfit(np.log(x), np.log(y), 1)[1]
            b = np.exp(ln_b)
            if 'add' in i:
                lst = []
                for j in x:
                    lst.append(b * (j ** k))
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] - data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2['total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] - data_agg_m2['total_ind']/12
                data_agg_m2['model_data'] = data_agg_m2['trend'] + data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['pow add'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('степенной')
                seas_lst.append('аддитивная')
            else:
                lst = []
                for j in x:
                    lst.append(b * (j ** k))
                data_agg_m2['trend'] = lst
                data_agg_m2['seasonality'] = data_agg_m2['turnoverOfGoods'] / data_agg_m2['trend']
                data_agg_m2['total_ind_seas'] = 0
                for j in range(12):
                    data_agg_m2.loc[(data_agg_m2.date == j+1), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                    data_agg_m2.loc[(data_agg_m2.date == j+13), 'total_ind_seas'] = data_agg_m2.loc[[j, j+12], 
                                                                                        'seasonality'].mean()
                data_agg_m2['total_ind'] = data_agg_m2.loc[:11, 'total_ind_seas'].sum()
                data_agg_m2['clear_ind_seas'] = data_agg_m2['total_ind_seas'] / (data_agg_m2['total_ind']/12)
                data_agg_m2['model_data'] = data_agg_m2['trend'] * data_agg_m2['clear_ind_seas']
                data_agg_m2['model_data'] = round(data_agg_m2['model_data'])
                models_dct['pow mul'] = r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])
                r2_lst.append(round(r2_score(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data']), 3))
                mape_lst.append(round(np.mean(np.abs((data_agg_m2["turnoverOfGoods"] - data_agg_m2['model_data']
                                                        ) / data_agg_m2["turnoverOfGoods"])) * 100, 3))
                mae_lst.append(int(mean_absolute_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                mse_lst.append(int(mean_squared_error(data_agg_m2['turnoverOfGoods'], data_agg_m2['model_data'])))
                rmse_lst.append(int(np.sqrt(mean_squared_error(data_agg_m2['turnoverOfGoods'], 
                                                                data_agg_m2['model_data']))))
                tr_lst.append('степенной')
                seas_lst.append('мультипликативная')
    if len(models_dct) != 1:  
        k = get_key(models_dct, max(models_dct.values()))
        dct = {k:0}    
        accuracy_dct = {'Тренд':tr_lst, 'Сезонность':seas_lst, 
                        'MAPE':mape_lst, 'MAE':mae_lst, 
                        'RMSE':rmse_lst, 'R^2':r2_lst}
        df_metrics = pd.DataFrame(data=accuracy_dct)
        find_model_predskaz(data, dct, df=df_metrics) 
        
    else:
        data_for_plot = data_agg_m2
        ls = nes_data2['date'].dt.strftime('%m/%Y').unique()
        ls = list(ls)
        data_for_plot['dates'] = ls
        data_for_plot['dates'] = data_for_plot['dates'].str.replace('2019', '2020')

        # построение прогноза
        data_lst = data_for_plot['model_data'].to_list()
        date_lst = data_for_plot['date'].to_list()
        trend_lst = data_for_plot['trend'].to_list()
        for i in range(data_agg_m2.loc[data_agg_m2.shape[0]-1,'date']+1, 
                        data_agg_m2.loc[data_agg_m2.shape[0]-1,'date']+1+n):
            date_lst.append(i)
            tr = k * i + b 
            trend_lst.append(tr)
            seas = data_agg_m2.loc[data_agg_m2.loc[data_agg_m2.shape[0]-1,
                                            'date']+1 - 13]['clear_ind_seas']
            data_lst.append(tr + seas)
        s_y = nes_data2['date'].max().year 
        s_m = nes_data2['date'].max().month
        start_date = DT.datetime(s_y, s_m, 1)
        if (s_m + n) % 12 > 0 and (s_m + n) % 12 < 12:
            e_y = s_y + 1
            e_m = (s_m + n) % 12
        else:
            e_y = s_y
            e_m = s_m + n
        end_date = DT.datetime(e_y, e_m, 1) 
        res = pd.date_range(
            min(start_date, end_date),
            max(start_date, end_date)
                                ).strftime('%m/%Y').tolist()
        dates_lst = ls + sorted(list(set(res[31:])))
        d = {'date':date_lst, 'turnover':data_lst, 'dates':dates_lst, 
            'trend':trend_lst}
        df_plt = pd.DataFrame(data=d)
        df_plt['dates'] = df_plt['dates'].str.replace('2019', '2020')
        fig = go.Figure()
        fig.add_trace(go.Scatter(name='Оборот фактический', x=data_for_plot['dates'], 
                                    y=data_for_plot['turnoverOfGoods'],
                                    mode='lines',
                                    line = dict(color='rgb(102, 194, 165)')))
        fig.add_trace(go.Scatter(name='Оборот предсказанный', x=df_plt['dates'], 
                                y=df_plt['turnover'],
                                mode='lines',
                                line = dict(color='rgb(102, 194, 165)', width=2, dash='dash')))

        fig.add_trace(go.Scatter(x=df_plt['dates'], y=df_plt['trend'],
                            mode='lines',
                            name='Тренд', line = dict(color='gray', width=1)))
        fig.update_layout( 
                        yaxis_title_text='Оборот, шт', 
                        showlegend = True, template="simple_white", 
                        height=600, width=1000, yaxis_tickformat = 'f.',     
                        legend=dict(yanchor="bottom", y=0.85, xanchor="right", x=0.3)
                        )
        if data['product_code'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота товара "{data["product_code"].unique()[0].replace("[", "").replace("]", "")}" по месяцам')
        elif data['subgroup'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота подгруппы товаров "{data["subgroup"].unique()[0].replace("[", "").replace("]", "")}" по месяцам')
        elif data['group'].nunique() == 1:
            fig.update_layout(title_text=f'Динамика величины оборота группы товаров "{data["group"].unique()[0].replace("[", "").replace("]", "")}" по месяцам')
        else:
            fig.update_layout(title_text=f'Динамика величины оборота категории товаров "{data["category"].unique()[0].replace("[", "").replace("]", "")}" по месяцам')
        fig.show()

        SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")
        
        df = df.rename(columns={'R^2':"R2".translate(SUP), 'MAPE':'MAPE, %', 'MAE':'MAE, шт', 'RMSE':'RMSE, шт'})
        df = df.sort_values(['R²', 'MAPE, %'], ascending=False).reset_index(drop=True)
        df = df.astype({'MAPE, %':'str', 'MAE, шт':'str', 'RMSE, шт':'str', 'R²':'str'})
        df['MAPE, %'] = df['MAPE, %'].str.replace('.', ',')
        df['MAE, шт'] = df['MAE, шт'].str.replace('.', ',')
        df['RMSE, шт'] = df['RMSE, шт'].str.replace('.', ',')
        df['R²'] = df['R²'].str.replace('.', ',')
        fig = go.Figure(data=[go.Table(
                header=dict(values=list(df.columns),
                            fill_color='rgb(102, 194, 165)',
                            align='center',
                            line_color='darkslategray', font=dict(color='black', size=12)),
                cells=dict(values=[df['Тренд'], df['Сезонность'], df['MAPE, %'], df['MAE, шт'], df['RMSE, шт'], df['R²']],
                        fill_color='white',
                        line_color='darkslategray',
                        align=['center', 'center', 'right', 'right', 'right', 'right'], font=dict(size=12)))
            ])
        fig.update_layout(title_text='Значения метрик точности предсказаний моделей')
        fig.show()


In [ ]:
find_model_predskaz(nes_data2, models_dct)

---

---
### Функция для наложения на ряд оборота отдельного товара модели подгруппы, или группы, или категории (если в качестве запроса – код товара)

на вход функции подается dataframe с данными величины оборота анализируемого товара, и dataframe с данными величины оборота всех товаров, вывод функции включает график, на котором изображается исходный ряд, ряд оборота подгруппы, группы или категории, ряд предсказанных значений оборота подгруппы, группы или категории, а также таблицу с рассчитанными значениями коэффициента корреляции рядов оборота товара и подгруппы, товара и группы, товара и категории

In [ ]:
def find_model_group(data, base):
    # расчет коэффициентов корреляции и построение модели 
    # ряда динамики оборота подгруппы, группы или категории товаров
    prod_group = data['group'].unique()[0]
    prod_subgroup = data['subgroup'].unique()[0]
    prod_category = data['category'].unique()[0]
    prod_code = data['product_code'].unique()[0]
    group_data = base[base['group'] == prod_group].reset_index(drop=True)
    subgroup_data = base[base['subgroup'] == prod_subgroup].reset_index(drop=True)
    category_data = base[base['category'] == prod_category].reset_index(drop=True)
    prod_data = base[base['product_code'] == prod_code].reset_index(drop=True)
    # данные по динамике товара
    data_agg_m = prod_data[['date', 'turnoverOfGoods']].resample('M', on='date') \
                                                    .sum().reset_index(drop=True)
    ls_time = [i for i in range(1, 25)]
    data_agg_m = pd.concat([data_agg_m[:12], data_agg_m[24:]])
    data_agg_m['date'] = ls_time
    data_agg_m = data_agg_m.reset_index(drop=True)
    # данные по динамике подгруппы товаров
    data_agg_m_subgroup = subgroup_data[['date', 'turnoverOfGoods']] \
                            .resample('M', on='date').sum().reset_index(drop=True)
    ls_time = [i for i in range(1, 25)]
    data_agg_m_subgroup = pd.concat([data_agg_m_subgroup[:12], 
                                        data_agg_m_subgroup[24:]])
    data_agg_m_subgroup['date'] = ls_time
    data_agg_m_subgroup = data_agg_m_subgroup.reset_index(drop=True)
    # данные по динамике группы товаров
    data_agg_m_group = group_data[['date', 'turnoverOfGoods']] \
                        .resample('M', on='date').sum().reset_index(drop=True)
    data_agg_m_group = pd.concat([data_agg_m_group[:12], data_agg_m_group[24:]])
    data_agg_m_group['date'] = ls_time
    data_agg_m_group = data_agg_m_group.reset_index(drop=True)
    # данные по динамике категории товаров
    data_agg_m_cat = category_data[['date', 'turnoverOfGoods']] \
                    .resample('M', on='date').sum().reset_index(drop=True)
    data_agg_m_cat = pd.concat([data_agg_m_cat[:12], data_agg_m_cat[24:]])
    data_agg_m_cat['date'] = ls_time
    data_agg_m_cat = data_agg_m_cat.reset_index(drop=True)
    # анализ корреляции в динамике
    d = {'Подгруппа':round(np.corrcoef(data_agg_m['turnoverOfGoods'], 
                            data_agg_m_subgroup['turnoverOfGoods'])[0][1], 3), 
        'Группа':round(np.corrcoef(data_agg_m['turnoverOfGoods'], 
                            data_agg_m_group['turnoverOfGoods'])[0][1], 3), 
        'Категория':round(np.corrcoef(data_agg_m['turnoverOfGoods'], 
                            data_agg_m_cat['turnoverOfGoods'])[0][1], 3)}
    ls = base['date'].dt.strftime('%m/%Y').unique()
    ls = list(ls)
    s_y = data['date'].max().year 
    s_m = data['date'].max().month
    start_date = DT.datetime(s_y, s_m, 1)
    if (s_m) % 12 > 0 and (s_m) % 12 < 12:
        e_y = s_y + 1
        e_m = (s_m) % 12
    else:
        e_y = s_y
        e_m = s_m
        end_date = DT.datetime(e_y, e_m, 1) 
        res = pd.date_range(
                min(start_date, end_date),
                max(start_date, end_date)
                                    ).strftime('%m/%Y').tolist()
    dates_lst = ls + sorted(list(set(res[31:])))
    for i in range(len(dates_lst)):
        if dates_lst[i][-2:] == '19':
            dates_lst[i] = dates_lst[i].replace('2019', '2020')
        else:
            continue
    if get_key(d, max(d.values())) == 'Подгруппа' and max(d.values()) >= 0.5:
        x = data_agg_m_subgroup['date'].to_list()
        y = data_agg_m_subgroup['turnoverOfGoods'].to_list()
        k = np.polyfit(x, y, 1)[0]
        b = np.polyfit(x, y, 1)[1]
        lst = []
        for j in x:
            lst.append(k * j + b)
        data_agg_m_subgroup['trend'] = lst
        data_agg_m_subgroup['seasonality'] = data_agg_m_subgroup['turnoverOfGoods'] \
                                                - data_agg_m_subgroup['trend']
        data_agg_m_subgroup['total_ind_seas'] = 0
        for j in range(12):
            data_agg_m_subgroup.loc[(data_agg_m_subgroup.date == j+1), 
                                'total_ind_seas'] = data_agg_m_subgroup.loc[[j, j+12], 
                                    'seasonality'].mean()
            data_agg_m_subgroup.loc[(data_agg_m_subgroup.date == j+13), 
                                    'total_ind_seas'] = data_agg_m_subgroup.loc[[j, j+12], 
                                    'seasonality'].mean()
        data_agg_m_subgroup['total_ind'] = data_agg_m_subgroup.loc[:11, 
                                                'total_ind_seas'].sum()
        data_agg_m_subgroup['clear_ind_seas'] = data_agg_m_subgroup['total_ind_seas'] \
                                                - data_agg_m_subgroup['total_ind']/12
        data_agg_m_subgroup['model_data'] = data_agg_m_subgroup['trend'] \
                                             + data_agg_m_subgroup['clear_ind_seas']
        data_agg_m_subgroup['model_data'] = round(data_agg_m_subgroup['model_data'])

        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(name='Оборот товарной подгруппы (предсказанный)', 
                                x=dates_lst, y=data_agg_m_subgroup['model_data'],
                                mode='lines', line = dict(color='rgb(102, 194, 165)', 
                                width=2, dash='dash')), secondary_y=True)
        fig.add_trace(go.Scatter(name='Оборот товарной подгруппы (фактический)', 
                                x=dates_lst, y=data_agg_m_subgroup['turnoverOfGoods'],
                                mode='lines', line = dict(color='rgb(102, 194, 165)')), 
                                secondary_y=True)
        fig.add_trace(go.Scatter(x=dates_lst, y=data_agg_m['turnoverOfGoods'],
                                mode='lines', name='Оборот товара (фактический)', 
                                line = dict(color='gray', width=2)), secondary_y=False)
        fig.update_layout(yaxis_title_text='Оборот, шт', 
                            showlegend = True, template="simple_white", 
                            height=600, width=1000, yaxis_tickformat = 'f.',     
                            legend=dict(yanchor="bottom", y=0.85, xanchor="right", x=0.8))
        fig.update_yaxes(title_text="Оборот товара, шт", secondary_y=False)
        fig.update_yaxes(title_text="Оборот подгруппы, шт", secondary_y=True)
        fig.update_layout(title_text=f'Динамика величины оборота товара "{prod_code}" и подгруппы товаров "{prod_subgroup}" по месяцам')
        fig.show()       
    elif get_key(d, max(d.values())) == 'Группа' and max(d.values()) >= 0.5:
        x = data_agg_m_group['date'].to_list()
        y = data_agg_m_group['turnoverOfGoods'].to_list()
        k = np.polyfit(x, y, 1)[0]
        b = np.polyfit(x, y, 1)[1]
        lst = []
        for j in x:
            lst.append(k * j + b)
        data_agg_m_group['trend'] = lst
        data_agg_m_group['seasonality'] = data_agg_m_group['turnoverOfGoods'] \
                                            - data_agg_m_group['trend']
        data_agg_m_group['total_ind_seas'] = 0
        for j in range(12):
            data_agg_m_group.loc[(data_agg_m_group.date == j+1), 
                                'total_ind_seas'] = data_agg_m_group.loc[[j, j+12], 
                                'seasonality'].mean()
            data_agg_m_group.loc[(data_agg_m_group.date == j+13), 
                                'total_ind_seas'] = data_agg_m_group.loc[[j, j+12], 
                                'seasonality'].mean()
        data_agg_m_group['total_ind'] = data_agg_m_group.loc[:11, 'total_ind_seas'].sum()
        data_agg_m_group['clear_ind_seas'] = data_agg_m_group['total_ind_seas'] \
                                                - data_agg_m_group['total_ind']/12
        data_agg_m_group['model_data'] = data_agg_m_group['trend'] \
                                        + data_agg_m_group['clear_ind_seas']
        data_agg_m_group['model_data'] = round(data_agg_m_group['model_data'])
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(name='Оборот товарной группы (предсказанный)', 
                                x=dates_lst, y=data_agg_m_group['model_data'],
                                mode='lines', line = dict(color='rgb(102, 194, 165)', 
                                width=2, dash='dash')), secondary_y=True)
        fig.add_trace(go.Scatter(name='Оборот товарной группы (фактический)', 
                                x=dates_lst, y=data_agg_m_group['turnoverOfGoods'],
                                mode='lines', line = dict(color='rgb(102, 194, 165)')), 
                                secondary_y=True)
        fig.add_trace(go.Scatter(x=dates_lst, y=data_agg_m['turnoverOfGoods'],
                                mode='lines', name='Оборот товара (фактический)', 
                                line = dict(color='gray', width=2)), secondary_y=False)
        fig.update_layout(yaxis_title_text='Оборот, шт', 
                            showlegend = True, template="simple_white", 
                            height=600, width=1000, yaxis_tickformat = 'f.',     
                            legend=dict(yanchor="bottom", y=0.85, xanchor="right", x=0.43))
        fig.update_yaxes(title_text="Оборот товара, шт", secondary_y=False)
        fig.update_yaxes(title_text="Оборот группы, шт", secondary_y=True)
        fig.update_layout(title_text=f'Динамика величины оборота товара "{prod_code}" и группы товаров "{prod_group}" по месяцам')
        fig.show()   
    elif get_key(d, max(d.values())) == 'Категория' and max(d.values()) >= 0.5:
        x = data_agg_m_cat['date'].to_list()
        y = data_agg_m_cat['turnoverOfGoods'].to_list()
        k = np.polyfit(x, y, 1)[0]
        b = np.polyfit(x, y, 1)[1]
        lst = []
        for j in x:
            lst.append(k * j + b)
        data_agg_m_cat['trend'] = lst
        data_agg_m_cat['seasonality'] = data_agg_m_cat['turnoverOfGoods'] \
                                        - data_agg_m_cat['trend']
        data_agg_m_cat['total_ind_seas'] = 0
        for j in range(12):
            data_agg_m_cat.loc[(data_agg_m_cat.date == j+1), 
                                'total_ind_seas'] = data_agg_m_cat.loc[[j, j+12], 
                                'seasonality'].mean()
            data_agg_m_cat.loc[(data_agg_m_cat.date == j+13), 
                                'total_ind_seas'] = data_agg_m_cat.loc[[j, j+12], 
                                'seasonality'].mean()
        data_agg_m_cat['total_ind'] = data_agg_m_cat.loc[:11, 
                                        'total_ind_seas'].sum()
        data_agg_m_cat['clear_ind_seas'] = data_agg_m_cat['total_ind_seas'] \
                                            - data_agg_m_cat['total_ind']/12
        data_agg_m_cat['model_data'] = data_agg_m_cat['trend'] \
                                        + data_agg_m_cat['clear_ind_seas']
        data_agg_m_cat['model_data'] = round(data_agg_m_cat['model_data'])
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(name='Оборот товарной категории (предсказанный)', 
                                x=dates_lst, y=data_agg_m_cat['model_data'],
                                mode='lines', line = dict(color='rgb(102, 194, 165)', 
                                width=2, dash='dash')), secondary_y=True)
        fig.add_trace(go.Scatter(name='Оборот товарной категории (фактический)', 
                                x=dates_lst, y=data_agg_m_cat['turnoverOfGoods'],
                                mode='lines', line = dict(color='rgb(102, 194, 165)')), secondary_y=True)
        fig.add_trace(go.Scatter(x=dates_lst, y=data_agg_m['turnoverOfGoods'],
                                mode='lines', name='Оборот товара (фактический)', 
                                line = dict(color='gray', width=2)), secondary_y=False)
        fig.update_layout(yaxis_title_text='Оборот, шт', 
                            showlegend = True, template="simple_white", 
                            height=600, width=1000, yaxis_tickformat = 'f.',     
                            legend=dict(yanchor="bottom", y=0.85, xanchor="right", x=0.43))
        fig.update_yaxes(title_text="Оборот товара, шт", secondary_y=False)
        fig.update_yaxes(title_text="Оборот категории, шт", secondary_y=True)
        fig.update_layout(title_text=f'Динамика величины оборота товара "{prod_code}" и категории товаров "{prod_category}" по месяцам')
        fig.show()   
    else:
        return ""
    df = pd.DataFrame(data=d, index=[0])
    df = df.astype({'Подгруппа':'str', 'Группа':'str', 'Категория':'str'})
    df['Подгруппа'] = df['Подгруппа'].str.replace('.', ',')
    df['Группа'] = df['Группа'].str.replace('.', ',')
    df['Категория'] = df['Категория'].str.replace('.', ',')
    fig = go.Figure(data=[go.Table(
            header=dict(values=list(df.columns),
                        fill_color='rgb(102, 194, 165)',
                        align='center',
                        line_color='darkslategray', font=dict(color='black', size=12)),
            cells=dict(values=[df['Подгруппа'], df['Группа'], df['Категория']],
                    fill_color='white',
                    line_color='darkslategray',
                    align=['right', 'right', 'right'], font=dict(size=12)))
        ])
    fig.update_layout(title_text=f'Корреляция динамики оборота товара {prod_code} с динамикой оборота подгруппы, группы и категории')
    fig.show()
    return res

In [ ]:
find_model_group(nes_data, data2)

['12/2021']

---

---
### Функция для проведения XYZ-анализа (если запрашивается категория товаров)

на вход функции подается dataframe с данными величины оборота анализируемой категории товаров, и dataframe с данными величины оборота всех товаров, вывод функции включает в себя таблицу с граничными значениями коэффициента вариации для групп X, Y, Z, таблицу распределения товаров по группам, диаграмму распределения товаров на уровне групп на X, Y, Z группы, и линейную диаграмму распределения групп товаров (согласно справочнику) на X, Y, Z группы

In [ ]:
def get_xyz_group(data, full_data):
    # XYZ-анализ товаров внутри категории   
    if data['product_code'].nunique() != 1:
        # отсеивание товаров (продающихся реже, чем 1 раз в месяц)
        my_data = data[data['date'].dt.year == 2021]
        prod_lst = my_data.groupby(['product_code', 'month'], 
                                        as_index=False) \
                                        .agg({'turnoverOfGoods':'sum'}) \
                            .groupby('product_code', as_index=False) \
                                .agg({'month':'count'})
        prod_lst = prod_lst[prod_lst['month'] == 12]['product_code'] \
                                                    .to_list()
        my_data = my_data.query('product_code in @prod_lst') \
                                        .reset_index(drop=True)
        # определение порогов для групп XYZ (по категориям)
        cat = data['category'].unique()[0]
        cat_data = full_data[full_data['category'] == cat] \
                                    .reset_index(drop=True)
        cat_data = cat_data[cat_data['date'].dt.year == 2021]
        prod_lst = cat_data.groupby(['product_code', 'month'], 
                        as_index=False) \
                        .agg({'turnoverOfGoods':'sum'}) \
                            .groupby('product_code', as_index=False) \
                                .agg({'month':'count'})
        prod_lst = prod_lst[prod_lst['month'] == 12]['product_code'] \
                                                    .to_list()
        cat_data = cat_data.query('product_code in @prod_lst') \
                                    .reset_index(drop=True)
        std_cat_df = cat_data.groupby(['product_code', 'month'], 
                        as_index=False) \
                        .agg({'turnoverOfGoods':'sum'}) \
                            .groupby('product_code', as_index=False) \
                                .agg({'turnoverOfGoods':'std'}) \
                                    .rename({'turnoverOfGoods':'std'}, axis=1)
        mean_cat_df = cat_data.groupby(['product_code', 'month'], 
                        as_index=False) \
                        .agg({'turnoverOfGoods':'sum'}) \
                            .groupby('product_code', as_index=False) \
                                .agg({'turnoverOfGoods':'mean'}) \
                                    .rename({'turnoverOfGoods':'mean'}, axis=1)
        cat_df = mean_cat_df.merge(std_cat_df, how='inner', on='product_code')
        cat_df = mean_cat_df.merge(std_cat_df, how='inner', on='product_code')
        cat_df['var'] = cat_df['std'] / cat_df['mean'] * 100
        x_lim = np.percentile(cat_df['var'], 33)
        y_lim = np.percentile(cat_df['var'], 66)
        # проведение XYZ-группировки товаров
        std_df = my_data.groupby(['category', 'group', 'subgroup', 
                            'product_code', 'month'], as_index=False) \
                            .agg({'turnoverOfGoods':'sum'}) \
                                .groupby(['category', 'group', 'subgroup', 
                                    'product_code'], as_index=False) \
                                    .agg({'turnoverOfGoods':'std'}) \
                                        .rename({'turnoverOfGoods':'std'}, axis=1)
        mean_df = my_data.groupby(['category', 'group', 'subgroup', 
                            'product_code', 'month'], as_index=False) \
                            .agg({'turnoverOfGoods':'sum'}) \
                                .groupby(['category', 'group', 
                                        'subgroup', 'product_code'], 
                                    as_index=False) \
                                    .agg({'turnoverOfGoods':'mean'}) \
                                        .rename({'turnoverOfGoods':'mean'}, 
                                        axis=1)
        df1 = mean_df.merge(std_df, how='inner', on=['category', 'group', 
                                                'subgroup', 'product_code'])
        df1['var'] = df1['std'] / df1['mean'] * 100
        x_group = df1[df1['var'] <= x_lim]['product_code'].to_list()
        y_group = df1[(df1['var'] > x_lim) & (df1['var'] <= y_lim)]['product_code'] \
                                                                    .to_list()
        z_group = df1[df1['var'] > y_lim]['product_code'].to_list()
        index_lst = ['X', 'Y', 'Z']
        var_lst = [f'V<={"{0:.0f}".format(x_lim)}', f'{"{0:.0f}".format(x_lim)}<V<={"{0:.0f}".format(y_lim)}', f'V>{"{0:.0f}".format(y_lim)}']
        number_prod = [len(x_group), len(y_group), len(z_group)]
        d = {'XYZ сектор':index_lst, 'Количество товаров':number_prod, 
            'Коэффициент вариации, %':var_lst}
        inf = pd.DataFrame(data=d)
        fig = go.Figure(data=[go.Table(
            header=dict(values=list(inf.columns),
                        fill_color='rgb(102, 194, 165)',
                        align='center',
                        line_color='darkslategray'),
            cells=dict(values=[inf['XYZ сектор'], inf['Количество товаров'], 
                        inf['Коэффициент вариации, %']],
                        fill_color='white',
                        line_color='darkslategray',
                        align=['center', 'right', 'center']))])
        fig.update_layout(title_text=f'Результаты XYZ-анализа по категории товаров "{cat.replace("[", "").replace("]", "")}"')
        fig.show()
        df1['XYZ'] = 0
        df1.loc[(df1['var'] <= x_lim), 'XYZ'] = 'X'
        df1.loc[((df1['var'] > x_lim) & (df1['var'] <= y_lim)), 'XYZ'] = 'Y'
        df1.loc[(df1['var'] > y_lim), 'XYZ'] = 'Z'
        df1 = df1[['category', 'group', 'subgroup', 'product_code', 'var', 'XYZ']]
        df1 = df1.rename(columns={'category':'Категория', 'group':'Группа', 
                                    'subgroup':'Подгруппа', 'product_code':'Код товара', 
                                    'var':'Коэффициент вариации, %', 'XYZ':'XYZ сектор'})
        df1['Категория'] = df1['Категория'].str.replace(']', '').str.replace('[', '')
        df1['Коэффициент вариации, %'] = pd.Series(["{0:.2f}" \
                    .format(val) for val in df1['Коэффициент вариации, %']])
        df1['Коэффициент вариации, %'] = df1['Коэффициент вариации, %'] \
                                            .str.replace(".", ",")
        fig = go.Figure(data=[go.Table(
            header=dict(values=list(df1.columns),
                        fill_color='rgb(102, 194, 165)',
                        align='center',
                        line_color='darkslategray'),
            cells=dict(values=[df1['Категория'], df1['Группа'], 
                                df1['Подгруппа'], df1['Код товара'],
                                df1['Коэффициент вариации, %'],      
                                df1['XYZ сектор']],
                    fill_color='white',
                    line_color='darkslategray',
                    align=['center', 'center', 'center', 'right', 
                            'right', 'center']))])
        fig.show()
        # проведение XYZ-группировки (групп товаров)
        std_df = my_data.groupby(['group', 'month'], as_index=False) \
                            .agg({'turnoverOfGoods':'sum'}) \
                                .groupby('group', as_index=False) \
                                    .agg({'turnoverOfGoods':'std'}) \
                                        .rename({'turnoverOfGoods':'std'}, axis=1)
        mean_df = my_data.groupby(['group', 'month'], as_index=False) \
                            .agg({'turnoverOfGoods':'sum'}) \
                                .groupby('group', as_index=False) \
                                    .agg({'turnoverOfGoods':'mean'}) \
                                        .rename({'turnoverOfGoods':'mean'}, axis=1)
        df = mean_df.merge(std_df, how='inner', on='group')
        df['var'] = df['std'] / df['mean'] * 100
        x_group1 = df[df['var'] <= x_lim]['group'].to_list()
        y_group1 = df[(df['var'] > x_lim) & (df['var'] <= y_lim)]['group'].to_list()
        z_group1 = df[df['var'] > y_lim]['group'].to_list()

        # интерпретация групп XYZ
        x_names_lst = list(my_data.groupby(['group', 'product_code'], as_index=False) \
                                        .agg({'turnoverOfGoods':'count'}) \
                                            .query('product_code in @x_group')['group'] \
                                            .value_counts().index)
        x_values_lst = list(my_data.groupby(['group', 'product_code'], as_index=False) \
                                        .agg({'turnoverOfGoods':'count'}) \
                                            .query('product_code in @x_group')['group'] \
                                            .value_counts().values)
        y_names_lst = list(my_data.groupby(['group', 'product_code'], as_index=False) \
                                        .agg({'turnoverOfGoods':'count'}) \
                                            .query('product_code in @y_group')['group'] \
                                            .value_counts().index)
        y_values_lst = list(my_data.groupby(['group', 'product_code'], as_index=False) \
                                        .agg({'turnoverOfGoods':'count'}) \
                                            .query('product_code in @y_group')['group'] \
                                            .value_counts().values)
        z_names_lst = list(my_data.groupby(['group', 'product_code'], as_index=False) \
                                        .agg({'turnoverOfGoods':'count'}) \
                                            .query('product_code in @z_group')['group'] \
                                            .value_counts().index)
        z_values_lst = list(my_data.groupby(['group', 'product_code'], as_index=False) \
                                        .agg({'turnoverOfGoods':'count'}) \
                                            .query('product_code in @z_group')['group'] \
                                            .value_counts().values)
        labels1 = [i.replace('[', '').replace(']', '') for i in x_names_lst]
        labels2 = [i.replace('[', '').replace(']', '') for i in y_names_lst]
        labels3 = [i.replace('[', '').replace(']', '') for i in z_names_lst]
        fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, 
                                                    {'type':'domain'}, 
                                                    {'type':'domain'}]])
        fig.add_trace(go.Pie(labels=labels1, values=x_values_lst, name="X", 
                                marker_colors=px.colors.qualitative.Set2), 1, 1)
        fig.add_trace(go.Pie(labels=labels2, values=y_values_lst, name="Y"), 1, 2)
        fig.add_trace(go.Pie(labels=labels3, values=z_values_lst, name="Z"), 1, 3)
        fig.update_traces(hole=.4, hoverinfo="label+percent")
        fig.update_layout(title_text=f'XYZ-распределение товаров категории "{my_data["category"].unique()[0].replace("[", "").replace("]", "")}"',
                                annotations=[dict(text='X', x=0.135, y=0.5, 
                                            font_size=20, showarrow=False),
                                            dict(text='Y', x=0.5, y=0.5, 
                                                font_size=20, showarrow=False),
                                            dict(text='Z', x=0.865, y=0.5, 
                                                font_size=20, showarrow=False)])
        fig.show()
        sort_df = df.sort_values('var')
        sort_df = sort_df.loc[sort_df['group'] != '[женские]колг.жен.']
        xl = sort_df[sort_df['var'] >= x_lim].iloc[0]['group']
        yl = sort_df[sort_df['var'] >= y_lim].iloc[0]['group']
        xcenter = sort_df.iloc[round(sort_df[sort_df['var'] <= x_lim].shape[0] / 2)]['group']
        ycenter = sort_df[(sort_df['var'] >= x_lim) & (sort_df['var'] < y_lim)] \
                        .iloc[round(sort_df[(sort_df['var'] >= x_lim) & (sort_df['var'] < y_lim)] \
                            .shape[0] / 2)]['group']
        zcenter = sort_df[sort_df['var'] >= y_lim].iloc[round(sort_df[sort_df['var'] >= y_lim] \
                                                .shape[0] / 2)]['group']
        xcenter = xcenter.replace('[', '').replace(']', '')
        ycenter = ycenter.replace('[', '').replace(']', '')
        zcenter = zcenter.replace('[', '').replace(']', '')
        sort_df['var'] = round(sort_df['var'], 2)
        fig1 = go.Figure()
        sort_df['group'] = sort_df['group'].str.replace('[', '') \
                                            .str.replace(']', '')
        fig1.add_trace(go.Scatter(x=sort_df['group'], y=sort_df['var'],
                                            mode='markers+lines', name='', 
                                            line = dict(color='rgb(27, 158, 119)', width=2)))
        fig1.update_layout(yaxis_title_text='Коэффициент вариации, %',
                            xaxis_title_text='Группа товаров',
                            showlegend=False, template="simple_white", 
                            height=600, width=1000, yaxis_tickformat = 'f.')
        fig1.add_vline(x=xl, line_width=2, line_dash="dash", line_color="black")
        fig1.add_vline(x=yl, line_width=2, line_dash="dash", line_color="black")
        fig1.add_trace(go.Scatter(
                        x=[xcenter, ycenter, zcenter],
                        y=[sort_df['var'].max(), sort_df['var'].max(), 
                        sort_df['var'].max()],
                        mode="text",
                        text=["Группа X", "Группа Y", "Группа Z"],
                        textposition="bottom center", textfont_size=16
                        ))
        fig1.update_layout(title_text=f'Распределение групп товаров категории "{cat.replace("[", "").replace("]", "")}" по группам XYZ')
        fig1.show()   
    else:
        return 'Невозможно провести XYZ-анализ'

In [ ]:
get_xyz_group(nes_data, data)

---

---
### Функция для кластеризации рядов оборота товаров (если заправшивается подгруппа, группа или категория товаров)

на вход функции подается dataframe с данными величины оборота анализируемой подгруппы, группы или категории товаров, вывод функции включает в себя таблицу распределения товаров по кластерам, диаграмму распределения товаров на уровне групп по кластерам и линейный график динамики средней величины оборота по кластерам

In [ ]:
def get_clusters(data):
    # кластеризация товаров внутри одной подгруппы, группы или категории
    if data['product_code'].nunique() != 1:
        # формирование датафрейма для кластеризации
        df_month = data.groupby('product_code')[['date', 'turnoverOfGoods']] \
                        .resample('M', on='date').sum().reset_index()
        df_month['date'] = df_month['date'].dt.month
        df_class_month = pd.pivot_table(df_month, values='turnoverOfGoods', 
                            index='product_code', columns='date').reset_index()
        df_class_month = df_class_month.fillna(0)
        # стандартизация данных
        scaler = StandardScaler()
        df_scaled_month = scaler.fit_transform(df_class_month.iloc[:,1:].T).T
        # выбор оптимального числа кластеров
        silhouette = []
        K = range(3, 9)
        for k in tqdm(K):
            kmeanModel = TimeSeriesKMeans(n_clusters=k, metric="euclidean", 
                                            n_jobs=6, max_iter=10)
            kmeanModel.fit(df_scaled_month)
            silhouette.append(silhouette_score(df_scaled_month, kmeanModel.labels_))
        k_opt = silhouette.index(max(silhouette)) + 3
        n_clusters = k_opt
        ts_kmeans = TimeSeriesKMeans(n_clusters=n_clusters, metric='euclidean', 
                                        n_jobs=3, max_iter=10)
        ts_kmeans.fit(df_scaled_month)
        # формирование датафрейма с отнесением товаров к кластерам
        df_class_month['cluster'] = ts_kmeans.predict(df_scaled_month)
        df = pd.DataFrame(df_class_month.groupby('cluster')['product_code'] \
                                        .value_counts())
        df_class_month = df_class_month.merge(data[['category', 'group', 
                                'subgroup', 'product_code']].drop_duplicates(), 
                                how='left', on='product_code')
        df_class_month = df_class_month[['category', 'group', 'subgroup', 
                                        'product_code', 'cluster']]
        df_class_month['cluster'] = df_class_month['cluster'] + 1
        df_class_month['category'] = df_class_month['category'].str.replace('[', '') \
                                                                .str.replace(']', '')
        df_class_month = df_class_month.rename(columns={'category':'Категория', 
                                                        'group':'Группа', 
                                                        'subgroup':'Подгруппа', 
                                                        'product_code':'Код товара', 
                                                        'cluster':'Кластер'})
        s_y = data['date'].max().year 
        s_m = data['date'].max().month
        start_date = DT.datetime(s_y, s_m, 1)
        if (s_m) % 12 > 0 and (s_m) % 12 < 12:
            e_y = s_y + 1
            e_m = (s_m) % 12
        else:
            e_y = s_y
            e_m = s_m
            end_date = DT.datetime(e_y, e_m, 1) 
            res = pd.date_range(
                    min(start_date, end_date),
                    max(start_date, end_date)
                                        ).strftime('%m/%Y').tolist()
        dates_lst = ls + sorted(list(set(res[31:])))
        dates_lst = dates_lst[-12:]
        # интерпретация кластеров
        df = df.rename(columns={'product_code':'n'})
        df2 = df.reset_index()
        df2 = df2.drop('n', axis=1)
        df_clus = df_month.merge(df2, how='left', on='product_code')
        fig = go.Figure(data=[go.Table(header=dict(values=list(df_class_month.columns),
                                                    fill_color='rgb(102, 194, 165)',
                                                    align='center',
                                                    line_color='darkslategray'),
                                        cells=dict(values=[df_class_month['Категория'], 
                                                            df_class_month['Группа'], 
                                                            df_class_month['Подгруппа'], 
                                                            df_class_month['Код товара'],
                                                            df_class_month['Кластер']],
                                                            fill_color='white',
                                                            line_color='darkslategray',
                                                            align=['center', 'center', 'center', 
                                                                    'right', 'right']))])
        fig.update_layout(title_text=f'Результаты кластерного анализа товаров категории 
                            "{data["category"].unique()[0].replace("[", "").replace("]", "")}"')                                                   
        fig.show()
        fig1 = go.Figure()
        for i in range(n_clusters):
            df_plt = df_clus.groupby(['date', 'cluster'], as_index=False) \
                            .agg({'turnoverOfGoods':'mean'})
            df_plt = df_plt[df_plt['cluster'] == i]
            df_plt['date'] = dates_lst
            fig1.add_trace(go.Scatter(x=df_plt['date'], y=df_plt['turnoverOfGoods'],
                                        mode='lines', name=f'Кластер {i+1}', 
                                        line = dict(width=2)))
        fig1.update_layout(title_text='Динамика средней величины оборота по кластерам',
                            yaxis_title_text='Оборот, шт',
                            showlegend=True, template="simple_white", 
                            height=600, width=1000, yaxis_tickformat = 'f.',   
                            legend=dict(yanchor="bottom", y=0.75, xanchor="right", x=0.2))
        fig1.show()   
        # распределение групп товаров по кластерам
        fig = go.Figure()
        df_clus = df_clus.merge(data[['product_code', 'group', 'subgroup']].drop_duplicates(), 
                                how='left', on='product_code')
        fig = make_subplots(rows=1, cols=n_clusters, specs=[[
                            {'type':'domain'} for i in range(n_clusters)]])
        for i in range(n_clusters):
            names_clus_lst = list(df_clus[df_clus['cluster'] == i].groupby(['group', 'product_code'], 
                                                    as_index=False) \
                                                    .agg({'turnoverOfGoods':'count'})['group'] \
                                                        .value_counts().index)
            values_clus_lst = list(df_clus[df_clus['cluster'] == i].groupby(['group', 'product_code'], 
                                                    as_index=False) \
                                                    .agg({'turnoverOfGoods':'count'})['group'] \
                                                        .value_counts().values)
            labels = names_clus_lst
            fig.add_trace(go.Pie(labels=labels, values=values_clus_lst, name=f"Кластер №{i+1}"), 1, i+1)
        fig.update_traces(hole=.4, hoverinfo="label+percent")
        if n_clusters == 3:
            fig.update_layout(title_text=f'Распределение групп товаров категории "{my_data["category"] \
                                            .unique()[0].replace("[", "").replace("]", "")}" по кластерам',
                                                annotations=[dict(text='Кластер №1', x=0.108, y=0.5, 
                                                font_size=14, showarrow=False),
                                                dict(text='Кластер №2', x=0.5, y=0.5, 
                                                font_size=14, showarrow=False),
                                                dict(text='Кластер №3', x=0.893, y=0.5, 
                                                font_size=14, showarrow=False)])
        fig.show()
        # распределение подгрупп товаров по кластерам
        fig = go.Figure()
        fig = make_subplots(rows=1, cols=n_clusters, 
                            specs=[[{'type':'domain'} for i in range(n_clusters)]])
        for i in range(n_clusters):
            names_clus_lst = list(df_clus[df_clus['cluster'] == i] \
                                .groupby(['subgroup', 'product_code'], as_index=False) \
                                .agg({'turnoverOfGoods':'count'})['subgroup'] \
                                .value_counts().index)
            values_clus_lst = list(df_clus[df_clus['cluster'] == i] \
                                .groupby(['subgroup', 'product_code'], as_index=False) \
                                .agg({'turnoverOfGoods':'count'})['subgroup'] \
                                .value_counts().values)
            labels = names_clus_lst
            fig.add_trace(go.Pie(labels=labels, values=values_clus_lst, 
                                name=f"Кластер №{i+1}"), 1, i+1)
        fig.update_traces(hole=.4, hoverinfo="label+percent")
        if n_clusters == 3:
            fig.update_layout(title_text=f'Распределение подгрупп товаров категории "{my_data["category"]
                                            .unique()[0].replace("[", "").replace("]", "")}" по кластерам',
                                            annotations=[dict(text='Кластер №1', x=0.108, y=0.5, 
                                            font_size=14, showarrow=False),
                                            dict(text='Кластер №2', x=0.5, y=0.5, 
                                                font_size=14, showarrow=False),
                                            dict(text='Кластер №3', x=0.893, y=0.5, font_size=14, 
                                                    showarrow=False)])
        fig.show()
    else:
        return "" 

In [ ]:
get_clusters(nes_data)

100%|██████████| 6/6 [00:00<00:00, 29.76it/s]
